# External QoiModel tutorial

In this tutorial you will learn the basics of the QoiModel interface. The QoIModel interface is almost identical to the model interface,
but has an additional method called compute_qoi() where it returns a (usually scalar) quantity of interest. This type of model is useful for various workflows, such as training a ROM to a desired tolerance on a specified QoI. The API for the model interface is provided here:

https://pressio.github.io/rom-tools-and-workflows/romtools/workflows/models.html


In [11]:
#First, let's import the relavant modules:
import romtools
import os
import numpy as np
from matplotlib import pyplot as plt
from romtools.workflows import models

In [1]:
# As a starting point, we will use the model created in the external_model tutorial
from ipynb.fs.full.external_model import adrExternalRomToolsModel

In [9]:
#Now, we will create a QoI model. As a QoI, we will take the gradient of the state at the right of the domain
class adrExternalRomToolsQoiModel(adrExternalRomToolsModel):
        # Our class will inherit all the methods from adrExternalRomToolsModel
    
    def compute_qoi(self, run_directory: str, parameter_sample: dict):
        # Note that compute_qoi is always called after run_model
        solution = np.load(run_directory + '/solution.npz')
        u = solution['u']
        x = solution['x']
        dx = x[1] - x[0] #we use a uniform grid
        ux_at_right_edge = (0. - u[-1])/dx
        return ux_at_right_edge

# We can now instatiate our QoI
myModel = adrExternalRomToolsQoiModel()

#That's it!

In [15]:
#Let's do an example run of the model similar to how it will be used in a workflow.

if __name__=="__main__":
    #First, let's make a dictionary that is a stand in for the parameter space
    parameter_sample = {}
    parameter_sample['c'] = 0.5
    parameter_sample['nu'] = 1e-1

    #Now, let's populate our run directory. In this example, we will just use the current run directory.
    #In a full workflow, romtools will manage creating directories
    myModel.populate_run_directory(os.getcwd(),parameter_sample)

    #Now, let's run the model
    run_dir = os.getcwd()

    myModel.run_model(run_dir,parameter_sample)

    qoi = myModel.compute_qoi(run_dir,parameter_sample)

    print('The QoI is ' + str(qoi))
    #We will finish by cleaning up the files we created
    os.system('rm params.dat')
    os.system('rm solution.npz')

The QoI is -6.429918802201725
